In [25]:
#load libraries
import numpy as np
from pulp import *
import pandas as pd

In [26]:
#question inputs
Products=['Ginko','Kola']
Locations=['Ethiopia','Tanzania','Nigeria']#plants
Cost_Ginko=[21,22.50,23] #in the 3 plants as above
Cost_Kola=[22.50,24.50,25.50]#in the 3 plants as above
Capacity=[425,400,750]#in the 3 plants as above
Min_Capacity=[100,250,600]
Demand=[550,650]#of the 2 drinks   
Fixed_Cost=[1500,2000,3000]

In [27]:
#lets rephrase the inputs to suit our LP modelling
Product_Demands=dict(zip(Products,Demand)) 
#using the format---> dict(zip(keys, values)) for a list keys & values
print Product_Demands
#similarly for capacity constraints
Location_Capacity=dict(zip(Locations,Capacity))
print Location_Capacity

Location_Fixed_Cost=dict(zip(Locations,Fixed_Cost))
print Location_Fixed_Cost

#now we have to get the Costs for each Product and at each Location
#in such a way that we can index it easily.
adummy=[Cost_Ginko,Cost_Kola]
#below line gives first index as Products and sec index as Locations
all_combos = makeDict([Products,Locations],adummy,0)#this shud work
#for example
print all_combos['Ginko']['Ethiopia']

#for the Minimum Capacity
Min_Cap=dict(zip(Locations,Min_Capacity))
print Min_Cap

{'Ginko': 550, 'Kola': 650}
{'Ethiopia': 425, 'Tanzania': 400, 'Nigeria': 750}
{'Ethiopia': 1500, 'Tanzania': 2000, 'Nigeria': 3000}
21
{'Ethiopia': 100, 'Tanzania': 250, 'Nigeria': 600}


In [28]:
#define  the problem
prob=LpProblem("nuts",LpMinimize)

#define the dec variables
x=LpVariable.dicts("dec_vars",(Products,Locations),lowBound=0,upBound=None)
#x can be accessed by x['Products'] or by x['Products']['Locations']
#example x["Kola"] gives the Kola at all 3 locations
#x["Kola"]['Nigeria'] gives Kola only at Nigeria Location

#Factory ON or OFF Decision variable
status=LpVariable.dicts("factory_status",(Locations),cat="Binary")

# The objective function is added to 'prob'
prob += lpSum([x[w][b]*all_combos[w][b] for w in Products for b in Locations])+lpSum([status[i]*Location_Fixed_Cost[i] for i in Locations])


#define the constraints
#demand cnstraint
for i in Products:
    prob +=lpSum(x[i])>=Product_Demands[i]

    
#MAX capacity constraint
for i in Locations:
    prob +=lpSum(x[j][i] for j in Products)<=Location_Capacity[i]

#MIN capacity constraint
for i in Locations:
    prob +=lpSum(x[k][i] for k in Products)>=[status[i]*Min_Cap[i]] 
    

#link binary dec_variables to the actual dec_Variables
for i in Locations:
    prob+=lpSum(x[j][i] for j in Products)<=1000*status[i]    
    
# solve the problem
status = prob.solve()
LpStatus[status]

'Optimal'

In [29]:
# print the results
print (value(prob.objective))
print ('\n')

print(prob)
for v in prob.variables():
    print(v.name, "=", v.varValue)

34425.0


nuts:
MINIMIZE
21*dec_vars_Ginko_Ethiopia + 23*dec_vars_Ginko_Nigeria + 22.5*dec_vars_Ginko_Tanzania + 22.5*dec_vars_Kola_Ethiopia + 25.5*dec_vars_Kola_Nigeria + 24.5*dec_vars_Kola_Tanzania + 1500*factory_status_Ethiopia + 3000*factory_status_Nigeria + 2000*factory_status_Tanzania + 0.0
SUBJECT TO
_C1: dec_vars_Ginko_Ethiopia + dec_vars_Ginko_Nigeria
 + dec_vars_Ginko_Tanzania >= 550

_C2: dec_vars_Kola_Ethiopia + dec_vars_Kola_Nigeria + dec_vars_Kola_Tanzania
 >= 650

_C3: dec_vars_Ginko_Ethiopia + dec_vars_Kola_Ethiopia <= 425

_C4: dec_vars_Ginko_Tanzania + dec_vars_Kola_Tanzania <= 400

_C5: dec_vars_Ginko_Nigeria + dec_vars_Kola_Nigeria <= 750

_C6: dec_vars_Ginko_Ethiopia + dec_vars_Kola_Ethiopia
 - 100 factory_status_Ethiopia >= 0

_C7: dec_vars_Ginko_Tanzania + dec_vars_Kola_Tanzania
 - 250 factory_status_Tanzania >= 0

_C8: dec_vars_Ginko_Nigeria + dec_vars_Kola_Nigeria
 - 600 factory_status_Nigeria >= 0

_C9: dec_vars_Ginko_Ethiopia + dec_vars_Kola_Ethiopia
 - 1000 